In [35]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.sparse import csr_matrix, csc_matrix, kron, eye, diags

In [114]:
# funtion that uses GGRF
def vector_rf(W, d, f_vec, p_h, node, random_walks = 100, h = 100):
    '''
    This funtion computes the feature vector of a node using GGRF
    Args:
        W: Adjacency matrix
        d: Degree vector
        f_vec: Function to compute modulation of the random walk
        p_h: Probability of stopping the random walk
        node: Node of interest
        random_walks: Number of random walks
        h: Default value
    Returns:
        phi: Feature vector of the node
    '''
    # Initial values
    n = h
    phi = np.zeros(len(d))
    m = random_walks
    f_m = f_vec(n)

    for w in range(m):
        # Initial values for the random walk
        load = 1.0
        current_node = node
        terminated = False
        walk_lenght = 0
        
        # Register of the nodes visited
        register = [current_node]
        #print(phi[current_node])
        counter = 0
        #print("Random walk: ", w)
        while terminated == False:
            
            # In case we require more values of f
            if walk_lenght == n:
                #print("Requerí mas valores de f")
                n = 2 * n
                f_m = f_vec(n)
                #print(len(f_m))

            # Update the feature vector
            #print(phi[current_node], load, f_m[walk_lenght])
            phi[current_node] += load * f_m[walk_lenght]
            #if walk_lenght == 3:
            # print(load * f_m[walk_lenght], phi[current_node])
            #print(phi)
            # Update the walk length
            walk_lenght += 1

            # Select the next node searching in the neighbors
            #print(current_node)
            #neighbors = np.nonzero(W[current_node])[0]
            neighbors = W[current_node].indices
            #print(neighbors)
            new_node = np.random.choice(neighbors)
            aux = []
            # If the node is already in the register, we search for a new one
            while new_node in register:
                aux.append(new_node)
                new_node = np.random.choice(neighbors)
                if len(aux) == len(neighbors):
                    break
            # If we tried all the neighbors, we select a random one
            if len(aux) == len(neighbors):
                new_node = np.random.choice(neighbors)

            # Update the load
            #print(current_node, new_node)
            #print(load, d[current_node], W[current_node, new_node])
            load = load * (d[current_node].item() / (1 - p_h))* W[current_node, new_node]
            #print(d[current_node] / (1 - p_h))
            #print(new_node)

            # Update the current node
            current_node = new_node

            # Update the register
            register.append(current_node)
            counter += 1

            # Check if the random walk is terminated
            terminated = (np.random.uniform(0,0.5) < p_h)
            if counter == 150:
                break
            #print(phi[node])
            #print(phi / m)

    return phi / m

In [100]:
m_prueba = np.eye(5)
m_prueba_s = csr_matrix(m_prueba)
non_zero_indices = m_prueba_s[2].indices
non_zero_indices

array([2], dtype=int32)

In [107]:
def compute_f_vector(f_alpha, n):
    '''
    This function computes the modulation function for a given alpha function and n
    according to the GGRF paper
    Args:
        f_alpha: Alpha function
        n: Number of values to compute
    Returns:
        f: Modulation function of length n
    '''
    alpha = f_alpha(n)
    f = np.zeros(n)

    # Initial values
    f[0] = np.sqrt(alpha[0])
    aux = 2 * f[0]

    f[1] = alpha[1] / aux

    f[2] = (alpha[2] - f[1]**2) / aux

    # Compute the rest of the values
    for i in range(3, n):
        suma = sum(f[i-p] * f[p] for p in range(1, i))
        f[i] = (alpha[i] - suma) / aux

    return f

In [68]:
# coefficients for a Laplacian kernel

def alpha_laplace(s, n, d = 1):
    '''
    This function computes the alpha function for a Laplacian kernel
    Args:
        s: Laplacian kernel parameter for regularization
        n: Number of values to compute
        d: Default value (power of the degree)
    Returns:
        alpha: Alpha function of length n
    '''
    alpha = np.ones(n)
    aux1 = 0
    aux2 = 1
    # Recurrent formula
    q = 1 / (1 + s**(-2))
    #q = 1

    for i in range(1, n):
        alpha[i] = ((d + aux1) / aux2) * q * alpha[i-1]
        aux1 += 1
        aux2 += 1

    return alpha

In [69]:
# function to compute the kernel matrix

def kernel_graph_random_features(Wx, dx, f_vec, Px, Qx, p_h, random_walks = 100, dt = np.float32):
    '''
    This function computes the kernel value using the random features method
    Args:
        Wx: Adjacency matrix (normalized)
        dx: Degree vector
        f_vec: Function to compute modulation of the random walk
        Px: Probability vector with arriving probabilities
        Qx: Probability vector with leaving probabilities
        p_h: Probability of stopping the random walk
        random_walks: Number of random walks
    Returns:
        K: Kernel value
    '''
    # Define the matrices to store the feature vectors
    K1 = np.zeros(Wx.shape, dtype = dt)
    K2 = np.zeros(Wx.shape, dtype = dt)

    # Iteration over the nodes
    for i in range(len(dx)):
        # Compute the feature vector for the node i
        phi1 = vector_rf(Wx, dx, f_vec, p_h, i, random_walks)
        phi2 = vector_rf(Wx, dx, f_vec, p_h, i, random_walks)
        for j in range(len(dx)):
            K1[i][j] = phi1[j]
            K2[i][j] = phi2[j]

    # Compute the estimation
    K = K1 @ K2.T

    return np.dot(Qx, np.dot(K, Px))

In [64]:
def count_trips_mibici(data_user, threshold = 5, complement = False):
    viajes_user = data_user.groupby([data_user[['Origen_Id', 'Destino_Id']].min(axis=1), data_user[['Origen_Id', 'Destino_Id']].max(axis=1)]).size().reset_index(name='counts')
    viajes_user.columns = ['Est_A', 'Est_B', 'counts']
    if not complement:
        viajes_user = viajes_user[viajes_user['counts'] >= threshold]
    else:
        viajes_user = viajes_user[viajes_user['counts'] < threshold]
    if viajes_user.empty:
        return None
    total = viajes_user['counts'].sum()
    viajes_user['prob'] = viajes_user['counts']/total
    viajes_user = viajes_user.sort_values(by = 'prob', ascending = False).reset_index(drop=True)
    return viajes_user

In [41]:
def leer_matriz(nombre_archivo):
    matriz = []
    with open(nombre_archivo, 'r') as archivo:
        archivo.readline()
        archivo.readline()
        for linea in archivo:
            fila = [float(valor) for valor in linea.strip().split()]
            matriz.append(fila)
    return matriz

def encontrar_estacion(est, matriz):
    for i in range(len(matriz)):
        if matriz[i][0] == est:
            return matriz[i][1], matriz[i][2]
    return None, None

In [42]:
def plot_counter(counter_user, est, title, save = False, dir = None):
    vertex = list(set(counter_user['Est_A'].unique().tolist() + counter_user['Est_B'].unique().tolist()))
    opacity = np.linspace(0.1, 0.5, len(counter_user))
    #print(vertex)
    plt.figure(figsize=(10, 6))
    for i in vertex:
        esta = encontrar_estacion(i, est)
        #print(esta)
        plt.scatter(esta[1], esta[0], color='blue')
        plt.text(esta[1] + 0.00001, esta[0] + 0.00001, str(i), fontsize=7, ha='left', va='bottom')
    for i in range(len(counter_user)):
        current_trip = counter_user.iloc[i]
        prob = current_trip["prob"]
        estA = current_trip["Est_A"]
        estB = current_trip["Est_B"]
        if estA == estB:
            plt.scatter(encontrar_estacion(estA, est)[1], encontrar_estacion(estA, est)[0], color='red', marker='*', s=100)
        else:
            aux = np.array([encontrar_estacion(estA, est), encontrar_estacion(estB, est)])
            plt.plot(aux[:,1], aux[:,0], color='black', alpha=opacity[i])
    plt.grid()
    plt.title(f'{title}')
    if save:
        directory = f'{dir}/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        plt.savefig(f'{directory}/{title}.png')
        plt.close()
    else:
        plt.show()

In [43]:
dir = '/home/user/Desktop/Datos/'
#dir = '/Users/antoniomendez/Desktop/Tesis/Datos/datos_limpios/'

In [44]:
data_2019 = pd.read_csv(f'{dir}mibici/2019.csv')
data = data_2019[data_2019['Inicio_del_viaje'].str.startswith('2019-01-01')]
data2 = data_2019[data_2019['Inicio_del_viaje'].str.startswith('2019-01-02')]
estaciones = leer_matriz(f'{dir}/Adj_mibici/matrices_estaciones/est_2019.txt')

In [45]:
counts_data = count_trips_mibici(data)
counts_data2 = count_trips_mibici(data2)

In [46]:
def compute_matrix(counter_user, normalized = False, self_loops = False):
    if not self_loops:
        counter_user = counter_user[counter_user['Est_A'] != counter_user['Est_B']]
    vertex = list(set(counter_user['Est_A'].unique().tolist() + counter_user['Est_B'].unique().tolist()))
    matrix = np.zeros((len(vertex), len(vertex)))
    for i in range(len(counter_user)):
        current_trip = counter_user.iloc[i]
        count = current_trip["counts"]
        estA = current_trip["Est_A"]
        estB = current_trip["Est_B"]

        matrix[vertex.index(estA)][vertex.index(estB)] = count
        matrix[vertex.index(estB)][vertex.index(estA)] = count
    if normalized:
        D = np.sum(matrix, axis = 1)
        D = np.diag(D)
        D = np.linalg.inv(np.sqrt(D))
        matrix = np.sqrt(D) @ matrix @ np.sqrt(D)
    return matrix

In [ ]:
m1 = compute_matrix(counts_data, normalized = True)
m2 = compute_matrix(counts_data2, normalized = True)

In [115]:
m1_s = csr_matrix(m1)
m2_s = csr_matrix(m2)
mx = kron(m1_s, m2_s, format='csr')

dx = np.sum(mx, axis = 1)

px = np.ones(len(dx)) / len(dx)
qx = np.ones(len(dx)) / len(dx)

f_alpha = lambda n: alpha_laplace(0.1, n, 1)

f_vec = lambda n: compute_f_vector(f_alpha, n)

p_h = 0.1

K = kernel_graph_random_features(mx, dx, f_vec, px, qx, p_h)
K

0.00024767359147076176

In [ ]:
def vector_rf(W, d, f_vec, p_h, node, random_walks=100, h=100):
    '''
    This funtion computes the feature vector of a node using GGRF
    Args:
        W: Adjacency matrix
        d: Degree vector
        f_vec: Function to compute modulation of the random walk
        p_h: Probability of stopping the random walk
        node: Node of interest
        random_walks: Number of random walks
        h: Default value
    Returns:
        phi: Feature vector of the node
    '''
    n = h
    phi = np.zeros(len(d), dtype=np.float32)
    m = random_walks
    f_m = f_vec(n).astype(np.float32)

    for w in range(m):
        load = np.float64(1.0)  # Temporary to avoid overflow
        current_node = node
        terminated = False
        walk_lenght = 0
        register = [current_node]
        counter = 0

        while not terminated:
            if walk_lenght == n:
                n = 2 * n
                f_m = f_vec(n).astype(np.float32)

            phi[current_node] += np.float32(load * f_m[walk_lenght])  # Convert again to float32

            walk_lenght += 1
            neighbors = W[current_node].indices
            new_node = np.random.choice(neighbors)
            aux = []
            while new_node in register:
                aux.append(new_node)
                new_node = np.random.choice(neighbors)
                if len(aux) == len(neighbors):
                    break
            if len(aux) == len(neighbors):
                new_node = np.random.choice(neighbors)

            # Calcular usando float64 y convertir el resultado a float32
            load = np.float64(load * (d[current_node].item() / (1 - p_h)) * W[current_node, new_node])

            current_node = new_node
            register.append(current_node)
            counter += 1

            terminated = (np.random.uniform(0, 0.5) < p_h)
            if counter == 150:
                break

    return phi / np.float32(m)


def compute_f_vector(f_alpha, n):
    '''
    Calcula la función de modulación para una función alpha dada y n
    '''
    alpha = f_alpha(n).astype(np.float32)
    f = np.zeros(n, dtype=np.float32)

    f[0] = np.sqrt(alpha[0])
    aux = 2 * f[0]
    f[1] = alpha[1] / aux
    f[2] = (alpha[2] - f[1]**2) / aux

    for i in range(3, n):
        suma = sum(f[i-p] * f[p] for p in range(1, i))
        f[i] = (alpha[i] - suma) / aux

    return f

def kernel_graph_random_features(Wx, dx, f_vec, Px, Qx, p_h, random_walks=100):
    '''
    Calcula el valor del kernel usando el método de random features
    '''
    K1 = np.zeros(Wx.shape, dtype=np.float32)
    K2 = np.zeros(Wx.shape, dtype=np.float32)

    for i in range(len(dx)):
        phi1 = vector_rf(Wx, dx, f_vec, p_h, i, random_walks).astype(np.float32)
        phi2 = vector_rf(Wx, dx, f_vec, p_h, i, random_walks).astype(np.float32)
        for j in range(len(dx)):
            K1[i][j] = phi1[j]
            K2[i][j] = phi2[j]

    K = K1 @ K2.T
    return np.dot(Qx, np.dot(K, Px).astype(np.float32))


In [121]:
mx_ = np.kron(m1, m2)

In [ ]:
m1_s = csr_matrix(m1)
m2_s = csr_matrix(m2)
mx = kron(m1_s, m2_s, format='csr')

dx = np.sum(mx, axis = 1)

px = np.ones(len(dx)) / len(dx)
qx = np.ones(len(dx)) / len(dx)

f_alpha = lambda n: alpha_laplace(0.1, n, 1)

f_vec = lambda n: compute_f_vector(f_alpha, n)

p_h = 0.3

K = kernel_graph_random_features(mx, dx, f_vec, px, qx, p_h)
K

0.00021280588223330404

In [ ]:
# bytes de la matriz dispersa mx
peso_mx = mx.data.nbytes

# bytes de la matriz densa mx_
peso_mx_ = mx_.nbytes

peso_mx/peso_mx_

0.0008926800238048007